In [1]:
from hazm import Normalizer, word_tokenize, Stemmer, Lemmatizer
import itertools, collections
import math, heapq
import pandas as pd
import pickle,json
from collections import defaultdict

In [2]:
normalizer = Normalizer()
stemmer = Stemmer()
lemmatizer = Lemmatizer()

N_documents = 12201
tokens = []
total_tf = {}

In [3]:
def check_punctuation(text):
    stop_words = []
    with open (r'stopwords.txt', 'r', encoding='utf_8') as file: 
        stop_words = file.read().splitlines()
        
    removed = []
    for t in text:
        flg = False
        for s in stop_words:
            if t == s:
                flg = True
                break
        if not flg:
            removed.append(t)
    return removed


In [4]:
class PreProcessor:
    def __init__(self) -> None:
        pass

    def process(self, text):
        normalized_content = normalizer.normalize(text=text)
        terms = word_tokenize(normalized_content)
        clear_content = check_punctuation(terms)
        stemmed_terms = []
        for t in clear_content:
                stemmed_terms.append(stemmer.stem(t))
        return stemmed_terms


In [5]:
data = pd.read_json(r'IR_data.json')
pre_processor = PreProcessor()

inverted_index_with_frequency = defaultdict(list)
print('hi')
for i in range (0, N_documents):
        content = data[i]['content']
        stemmed_terms = pre_processor.process(content)
        in_doc_tokens = []
        in_doc_tf = {}
        for t in stemmed_terms:
                in_doc_tokens.append({'term': t, 'docID': i})
                in_doc_tf[t] = in_doc_tf.get(t,0) + 1
        tokens.extend(in_doc_tokens)

        # As said before instead of vector we record weight of each term in doc
        total_tf[i] = collections.OrderedDict(sorted(in_doc_tf.items()))
        for t in total_tf[i]:
                inverted_index_with_frequency[t].append({i:in_doc_tf[t]})

##print(dictonary['آسیا'])

hi


In [6]:
inverted_index = collections.defaultdict(list)
for token in tokens:
    values = inverted_index.get(token['term'],[])
    if token['docID'] not in values:
        values.append(token['docID'])
    inverted_index[token['term']] = values   
##inverted_index = collections.OrderedDict(sorted(inverted_index.items()))
  


stop_words = []
for term in inverted_index.keys():
    if len(inverted_index[term]) > 0.3 * N_documents:
        stop_words.append(term)
for s in stop_words:
    inverted_index.pop(s)

In [7]:
#idf
idf = {}
for term in inverted_index:
    nf = len(inverted_index[term])
    idf[term] = math.log(N_documents / nf, 10)

In [8]:
## Query Vectorize and Index Elimination
pre_processor = PreProcessor()

# with open ("inverted_index.json", "w") as f:
# json.dump(inverted_index,f)

query = input("Enter Yout Query: ")
splited_query = pre_processor.process(query)
print(splited_query)

term_freq = dict(collections.Counter(splited_query))
print(term_freq)
query_term_tf = {}
for term, freq in term_freq.items():
        if term in inverted_index.keys():
            query_term_tf[term] = 1+math.log(freq,10)


['مرد', 'سالار']
{'مرد': 1, 'سالار': 1}


In [9]:
print(idf['فایزر'])
for doc, doc_tf in total_tf.items():
        for d_term in doc_tf.keys():
            if d_term == "فایزر":
                if doc_tf[d_term] > 0:
                    # print('Found {} in doc {}'.format(d_term, doc))
                    print(1+math.log(doc_tf[d_term], 10))

3.6092741724045876
1.4771212547196624
1.0
1.0


In [10]:
docs_score = {}
for q_term, freq in query_term_tf.items():
    for doc, doc_tf in total_tf.items():
        for d_term in doc_tf.keys():
            if  d_term == q_term:
                # print('Found {} in doc {}'.format(d_term, doc))
                wd = (1+math.log(doc_tf[d_term], 10))*(idf[d_term])
                print(wd)
                docs_score[doc] = docs_score.get(doc,0) + (freq * wd)
print(docs_score)

2.045002741966025
2.045002741966025
2.045002741966025
2.045002741966025
2.045002741966025
2.045002741966025
2.045002741966025
2.045002741966025
2.660609908512887
2.660609908512887
2.045002741966025
2.045002741966025
2.660609908512887
3.0207170161180046
2.660609908512887
2.045002741966025
2.045002741966025
2.045002741966025
2.045002741966025
2.045002741966025
2.045002741966025
2.660609908512887
2.045002741966025
2.045002741966025
2.045002741966025
2.045002741966025
2.045002741966025
3.0207170161180046
2.045002741966025
2.045002741966025
2.045002741966025
2.045002741966025
2.045002741966025
2.045002741966025
2.045002741966025
2.045002741966025
2.045002741966025
2.045002741966025
2.660609908512887
2.045002741966025
2.660609908512887
2.045002741966025
2.045002741966025
2.045002741966025
2.660609908512887
2.045002741966025
2.045002741966025
2.660609908512887
3.276217075059749
2.045002741966025
2.045002741966025
2.045002741966025
2.045002741966025
2.045002741966025
2.045002741966025
2.045002

In [11]:
def get_doc_length(doc):
    l = 0
    for term, tf in total_tf[doc].items():
        if term in inverted_index.keys():
            l += (idf[term] * (1+math.log(tf, 10)))**2
    return math.sqrt(l)

In [12]:
## Similarity
for doc, score in docs_score.items():
    docs_score[doc] = score/get_doc_length(doc=doc)

print(docs_score)

{277: 0.049955065159919446, 278: 0.06042465543361134, 403: 0.14061044399812067, 679: 0.08621501616734108, 732: 0.09196340240174954, 785: 0.10755734679457103, 800: 0.11611346586665122, 974: 0.04892965791374303, 1834: 0.15159696221522423, 2880: 0.14930526269689748, 3299: 0.22114561460081897, 3446: 0.06469894924483113, 3842: 0.16821457013847452, 4140: 0.11573685246960361, 4243: 0.14187261018023684, 4743: 0.04848078715061959, 4828: 0.062045273991535775, 4911: 0.07416751122023772, 5036: 0.183088536256268, 6407: 0.11698219220374903, 6650: 0.11991721228083979, 6846: 0.07017558628133105, 6881: 0.07482344989086251, 7196: 0.06079582478566409, 7217: 0.06686628942128109, 7231: 0.08078270531189279, 7433: 0.08496601057540604, 7446: 0.1805738325842435, 7453: 0.11369361824699643, 7669: 0.08703719356617398, 7790: 0.06913009805859237, 7806: 0.04939294541434891, 7848: 0.05568345791849261, 7849: 0.04852079521340419, 7855: 0.04365388880107347, 7890: 0.06128573518108366, 8024: 0.09304680111106435, 8048: 0.1

In [13]:
# Return Top-K using heap method
heap = [(-value, key) for key,value in docs_score.items()]
largest = heapq.nsmallest(5, heap)
top_k = [(y, -x) for x,y in largest]

print(top_k)

[(3299, 0.22114561460081897), (10011, 0.19488746942917765), (5036, 0.183088536256268), (7446, 0.1805738325842435), (3842, 0.16821457013847452)]


In [14]:

champion_list = {}
for term in inverted_index_with_frequency.keys():
    document = {}
    for d in inverted_index_with_frequency[term]:
        document.update(d)
    sorted_ch = {k: v for k,v in sorted(document.items(), key=lambda item: item[1], reverse=True)}
    sorted_ch = dict(itertools.islice((sorted_ch.items()), 100))
    # print(sorted(sorted_result.items()))
    champion_list[term] = collections.OrderedDict((sorted_ch.items()))

In [15]:

docs_score_ch = {}
print(query_term_tf)
for q_term, q_freq in query_term_tf.items():
    for doc, freq in champion_list[q_term].items():
        print(freq, doc)
        wd = (1+math.log(freq, 10))*(idf[q_term])
        docs_score_ch[doc] = docs_score_ch.get(doc,0) + (q_freq * wd)

for doc, score in docs_score_ch.items():
    docs_score_ch[doc] = score/get_doc_length(doc=doc)

heap = [(-value, key) for key,value in docs_score_ch.items()]
largest = heapq.nsmallest(5, heap)
top_k = [(y, -x) for x,y in largest]

print(top_k)


{'سالار': 1.0}
5 10287
4 9078
4 10756
4 11857
4 12109
3 4140
3 7446
3 10011
3 10116
3 10153
3 10285
3 10418
3 11070
3 11405
2 1834
2 2880
2 3842
2 4243
2 6846
2 8085
2 8272
2 8894
2 9074
2 9496
2 9671
2 9837
2 10049
2 10295
2 10428
2 10492
2 10759
2 10900
2 11075
2 11324
2 11802
2 11878
2 12048
1 277
1 278
1 403
1 679
1 732
1 785
1 800
1 974
1 3299
1 3446
1 4743
1 4828
1 4911
1 5036
1 6407
1 6650
1 6881
1 7196
1 7217
1 7231
1 7433
1 7453
1 7669
1 7790
1 7806
1 7848
1 7849
1 7855
1 7890
1 8024
1 8048
1 8162
1 8281
1 8625
1 8852
1 9051
1 9071
1 9110
1 9271
1 9291
1 9294
1 9384
1 9396
1 9403
1 9535
1 9576
1 9695
1 9712
1 9768
1 9871
1 10286
1 10313
1 10473
1 10651
1 10659
1 10790
1 10801
1 10819
1 10844
1 10870
1 11078
1 11090
1 11099
[(3299, 0.22114561460081897), (10011, 0.19488746942917765), (5036, 0.183088536256268), (7446, 0.1805738325842435), (3842, 0.16821457013847452)]
